In [122]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
import threading
import time
import PyPDF2
import urllib.request
import io
import os
from collections import deque
import traceback
import sys
import logging
import re
import socket
import pyautogui

In [123]:
def clean_folder_name(folder_name):
    # Define a regex pattern for illegal characters in Windows folder names
    illegal_chars = r'[<>:"\\|?*\x00-\x1f]'
    # Replace illegal characters with an empty string
    cleaned_name = re.sub(illegal_chars, '', folder_name)
    return cleaned_name

In [124]:

try:
    os.makedirs("Logs")
except:
    pass

logfile = "Logs/" + "Log" + str(len(os.listdir("Logs"))+1)+".log"
class Tee:
    def __init__(self, *files):
        self.files = files

    def write(self, obj):
        for f in self.files:
            f.write(obj)

    def flush(self):
        for f in self.files:
            f.flush()

def log_print(*args, log_level=logging.INFO, sep=' ', end='\n', file=None, flush=False):
    # Convert args to a single string
    msg = sep.join(map(str, args)) + end
    
    # Log the message only if it's coming from log_print
    if file is log_file:
        logger.log(log_level, msg)

    # log_print to stdout
    if file is None:
        file = sys.stdout
    print(msg, end='', file=file, flush=flush)

# Create a logger specifically for log_print messages
log_file = logging.getLogger("log_print")
log_file.setLevel(logging.DEBUG)

# Create a file handler for log_print logger
file_handler = logging.FileHandler(logfile)
file_handler.setLevel(logging.DEBUG)

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the handler to the logger
log_file.addHandler(file_handler)

# Redirect sys.stdout to the log_print logger's stream
sys.stdout = Tee(sys.stdout, file_handler.stream)


In [125]:
visited = set([])

with open("banlist.txt", 'r') as file:
    banList = set([line.strip(" \n") for line in  file.readlines()])


In [126]:
download_folder_path = r"C:\Users\thaku\OneDrive\Desktop\Code\Stratagile\CompleteScrapper\Downloads"

In [127]:
def makeDriver():
    browser_options = ChromeOptions()
    browser_options.headless = True
    browser_options.add_argument("--start-maximized")
    driver = Chrome(options= browser_options)
    driver.implicitly_wait(10)
    return driver

def makeDownloadDriver():
    browser_options = ChromeOptions()
    browser_options.headless = True
    browser_options.add_experimental_option("prefs", {
        "download.default_directory": download_folder_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False,
    })
    browser_options.add_argument("--start-maximized")
    driver = Chrome(options= browser_options)
    driver.implicitly_wait(10)
    return driver

In [128]:
def waitForBoxPopup(driver):
    try:
        close_button = WebDriverWait(driver,5).until(
            EC.presence_of_element_located((By.XPATH,"//button[@id='qa-modal-close']"))
        )
        close_button.click()
        log_print("Closed the pop-up")
    except TimeoutException:
        log_print("No pop-up appeared")
    except NoSuchElementException:
        log_print("Close button not found")
    except:
        log_print("POPUP exception")

    try:
        element_to_delete = driver.find_element(By.XPATH, '//*[@id="qa-app-base"]/span')
        driver.execute_script("arguments[0].remove();", element_to_delete)
    except:
        log_print("Form exception")

In [129]:
def empty_folder(folder_path):
    # Iterate over all the files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if the file is a directory
        if os.path.isdir(file_path):
            # Recursively call empty_folder function for subdirectories
            empty_folder(file_path)
        else:
            # Remove the file
            os.remove(file_path)

In [130]:
def preProccessURL(url):
    url = url.strip(" /")
    url = url[:(url.find(".html#")+len(".html") if  url.find(".html#")!=-1 else None)]
    return url

def IsBanned(url):
    print("Testing ban for URL: ",url)
    try:
        if(url is None or type(url)!=str):
            log_print("BAN0")
            return True
        if url.find("screener.in")== -1:
            log_print("BAN1")
            return True
        if not url.startswith("https://www.screener.in/company") and not url == "https://www.screener.in/explore":
            log_print("BAN2")
            return True
        for item in banList:
            if url.find(item)!=-1:
                log_print("BAN3 URL ",url," is banned!")
                return True
            
    except:
        log_print("Error in preProccessing of url: ",url)
        return True
    return False

def getBaseFolder(url,baseFolder):
    if(url.startswith("https://www.screener.in/")):
            baseFolder = url[(url.find(".in/")+len(".in/")):(url.rfind(".html") if url.rfind(".html")!=-1 else None)]
            log_print("Set BaseFolder to: ", baseFolder)
    return baseFolder

In [131]:
def main(rootURL, BaseFolder, mode = 'Continue',baseFolder = ""):
    if mode== 'Continue':
        log_print("CONTINUING FROM PREV EXIT \n")
    else:
        baseFolder = ""
    global visited
    driver = makeDriver()
    rootURL = rootURL.strip(" /")
    driver.get(rootURL)
    time.sleep(60)

    dq = deque()
    with open("visited.txt",('r' if mode == 'Continue' else 'w'), encoding='utf-8') as file:
        if mode == 'Continue':
            visited = set([line.strip(" \n") for line in  file.readlines()])
    with open('queue.txt', ('r' if mode == 'Continue' else 'w'), encoding='utf-8') as file:
        if mode == 'Continue':
            for line1 in [line.strip(" \n") for line in  file.readlines()]:
                dq.append(line1)
    # print(type(visited))
    # log_print('TRUE' if visited == None else 'FALSE')
    if mode!='Continue':
        dq.append(rootURL)
    log_print(list(dq))
    try:
        while(len(dq)>0):
            currURL = dq.popleft()
            time.sleep(2)
            log_print("1, currURL: ",currURL)
            #PREPROCESSING
            try:
                url = preProccessURL(currURL)
                if IsBanned(url) or url in visited:
                    continue
                visited.add(url)
                with open("visited.txt",'a', encoding='utf-8') as file:
                    file.write(url+"\n")
                baseFolder = getBaseFolder(url,baseFolder)
            except Exception as e:
                log_print("Error in preproccessing! URL: ",url)
                log_print(traceback.format_exc())
                continue
            log_print("2, currURL: ",currURL)




            #OPENING/CLOSING OF DRIVER
            try:
                log_print("Opened URL: ",url," Base Folder: ",baseFolder)
                driver.get(url)
                time.sleep(2)
                log_print("3, currURL: ",currURL)
                if(url.startswith("https://www.screener.in/company/compare")):
                    try:
                        Catagory_name = driver.find_elements(By.XPATH, "/html/body/main/div[2]/h1")[0].get_attribute("h1")
                        log_print("Changed baseFolder to ",Catagory_name)
                        baseFolder = Catagory_name
                        table = driver.find_element(By.XPATH,"/html/body/main/div[2]/div[3]/table")
                        inpage_links = [i for i in [link.get_attribute("href") for link in table.find_elements(By.TAG_NAME,"a")] if not i.startswith("https://www.screener.in/company/compare")]
                        for page in inpage_links:
                            driver.get(page)
                            time.sleep(3)
                            try:
    # Find the button using the specified XPath
                                button = driver.find_element(By.XPATH,'//*[@id="top"]/div[1]/form/button')
                                try:
                                    # Click the button
                                    button.click()
                                    print("Button clicked successfully!")
                                    time.sleep(2)
                                except Exception as e:
                                    print("Error:", e)
                            except NoSuchElementException:
                                print("Error: NoSuchElementException occurred. The button could not be found.")
                        continue
                    except:
                        log_print(traceback.format_exc)
                
            except Exception as e:
                log_print("Error in Opening/closing driver, URL: ",url)
                log_print(traceback.format_exc())
                continue
            log_print("HOGAYA")
            # return
            #TODO PAGE INFORMATION EXTRACTOR
              

            #LINK EXCTRACTION AND BFS CONTINUIATION
            try:
                links = [link.get_attribute("href") for link in [i for i in driver.find_elements(By.TAG_NAME,"a") if i is not None] if link.get_attribute("href") is not None]
                log_print("Links: ")
                log_print(*links,sep="\t")
                for link in links:
                    try:
                        tlink = preProccessURL(link)
                    except:
                        log_print("Error in BFS QUEUE preprocessingof link ",link,". URL: ",url)
                        log_print(traceback.format_exc())
                        continue
                    if not IsBanned(tlink) and tlink not in visited:
                        dq.append(tlink)
                        with open('queue.txt','w', encoding='utf-8') as file:
                            for entry in list(dq):
                                file.write(str(entry) + "\n")
            except:
                log_print("Error in Link Extraction")
                log_print(traceback.format_exc())
                continue

            
    except:
        log_print(traceback.format_exc())
        log_print("Caution, an Error was thrown!")
    
    driver.close()
   
def removeClass(links, className):
    links2 = [link for link in links if link.get_attribute(className)!="hf-link"]
    return links2

In [132]:
# main(f"https://www.hpe.com/us/en/home.html","Extracted",mode='Continue',baseFolder="us/en/hpe-greenlake-compute")
main(f"https://www.screener.in/explore/","Extracted",mode='StartOver')